In [20]:
##
import os
import sys
import warnings
import enum
from enum import Enum
import matplotlib
from pathlib import Path
from typing import Callable, Dict, List

##
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

##
from finrl import config as finrl_config
from finrl.main import check_and_make_directories
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.config import (
    TEST_END_DATE,
    TRAINED_MODEL_DIR,
)
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.plot import backtest_stats, convert_daily_return_to_pyfolio_ts, get_baseline

##
import yfinance as yf

##
from meta.data_processors.yahoofinance import Yahoofinance

In [21]:
def config():
    ##
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)

    ##
    matplotlib.use("Agg")


config()

In [22]:
##
import pandas as pd
import tqdm
from dotenv import load_dotenv
import financedatabase as fd
import fundamentalanalysis as fa

##
from common.utils import now_time
from configuration.settings import ProjectDir
from rl.data.financial_modeling_prep import download
from rl.data.Company import Company

In [23]:
prj_dir = ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing"))

_TRAIN_DATA_START = "2000-01-01"
_TRAIN_DATA_END = "2015-12-31"
_TEST_DATA_START = "2016-01-01"
_TEST_DATA_END = "2021-12-31"

In [52]:
load_dotenv(prj_dir.root.joinpath("env/.env"))

try:
    print(os.getenv("EOD_HISTORICAL_DATA_API"))
    print(os.getenv("NASDAQ_HISTORICAL_DATA_API"))
    print(os.getenv("ALPHA_VANTAGE_API"))
    print(os.getenv("FINANCIAL_MODELING_PREP_API"))
except:
    raise ValueError("ERROR: API keys")

,2022-09,2022-06,2022-03,2021-12,2021-09,2021-06,2021-03,2020-12,2020-09,2020-06,...,2008-06,2008-03,2007-12,2007-09,2007-06,2007-03,2006-12,2006-09,2005-12,2004-12
reportedCurrency,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
cik,0001367644,0001367644,0001367644,0001367644,0001367644,0001367644,0001367644,0001367644,0001367644,0001367644,...,0001367644,0001367644,0001367644,0001367644,0001367644,0001367644,0001367644,0001367644,0001367644,0001367644
fillingDate,2022-11-09,2022-08-02,2022-04-29,2022-02-25,2021-11-05,2021-07-30,2021-04-30,2021-02-19,2020-11-06,2020-07-31,...,2008-08-07,2008-05-07,2008-03-10,2007-11-05,2007-08-07,2007-05-10,2006-12-31,2006-09-30,2005-12-31,2004-12-31
acceptedDate,2022-11-08 18:19:51,2022-08-01 18:02:14,2022-04-28 17:44:29,2022-02-25 16:35:23,2021-11-05 16:51:06,2021-07-29 19:08:30,2021-04-29 18:01:26,2021-02-18 18:52:37,2020-11-05 18:54:40,2020-07-31 16:52:20,...,2008-08-07 11:54:53,2008-05-07 16:17:36,2008-03-07 17:41:17,2007-11-02 20:34:49,2007-08-07 14:59:08,2007-05-10 16:58:55,2006-12-31,2006-09-30,2005-12-31,2004-12-31
calendarYear,2022,2022,2022,2021,2021,2021,2021,2020,2020,2020,...,2008,2008,2007,2007,2007,2007,2006,2006,2005,2004
period,Q3,Q2,Q1,Q4,Q3,Q2,Q1,Q4,Q3,Q2,...,Q2,Q1,Q4,Q3,Q2,Q1,Q4,Q3,Q4,Q4
cashAndCashEquivalents,240900000,358100000,435800000,576100000,403800000,447500000,547800000,621300000,415000000,268800000,...,84007000,92747000,105730000,24266000,33980000,67645000,76418000,19906000,36294000,6821000.0
shortTermInvestments,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190000.0,0.0,0.0
cashAndShortTermInvestments,240900000,358100000,435800000,576100000,403800000,447500000,547800000,621300000,415000000,268800000,...,84007000,92747000,105730000,24266000,33980000,67645000,76418000,20096000,36294000,6821000.0
netReceivables,191300000,175000000,181800000,274700000,254600000,261900000,184400000,230900000,196100000,258600000,...,32451000,25018000,18817000,54999000,34184000,7190000.0,44200000,6815000.0,3293000.0,18637000


idk
idk2
